In [3]:
import pandas as pd
import numpy as np

# Calculating sampling target sizes

**Step 1:** With the population counts aggregated in the previous step, we will need to calculate the sample size need to achieve adequite statistical power, using experience of harrassment as the critical variable. We will be using [Cochran's formula](https://www.tarleton.edu/academicassessment/documents/samplesize.pdf) to calculate appropriate sample sizes:

\begin{equation}
n_0=\frac{Z^2pq}{e^2}
\end{equation}
<div style="padding-left: 40%; font-size:12px">
where:<br>
    p = 0.215 (incidence of critical variable)<br>
    q = 1 - p<br>
    e = 0.05 (margin of error)<br>
    Z<sup>2</sup> = 1.96 (for 95% confidence level)
</div><br>

**Step 2:** n<sub>0</sub> needs to be further modified for each project to adjust the sample to the project's population, referred to as Finite Population Correction For Proportions, can achieved using the following equation:

\begin{equation}
n = \frac{n_0}{1 + \frac{(n_0 - 1)}{N}}
\end{equation}
<div style="padding-left: 40%; font-size:12px">
where: N = population size of a strata
</div><br>

**Step 3:** The sample size caclulated for each projects needs to be divided across various editing activity levels. This will be according to the proportions of the population. For example, if the desired sample size for arwiki is 117, and in the population, if 10% of the editors belong to 10-29 strata, the sample size for 10-29 strata on arwiki would be 12.

**Step 4:** Based on previous year's response rates, the sample size needs needs to be adjusted to account for nonresponse. This is capped at strata-level population). For example, 49 of the most-active (1200+) editors on en.wikipedia, and the 2020 response rate was 29%, we'd expect to need to contact 170 such contributors to achieve our target. 

### Step 1: Calculating n<sub>0</sub>

In [1]:
p = 0.215 # indence of critical variable (% experience harrasment)
e = 0.05 # margin of error
Z_square = 1.96 # 95% confidence

nO = (Z_square**2 * p * (1-p))/e**2 #Cochran's formula
print(nO)

259.3464159999999


### Step 2: Adjusting the sample to project's population

In [4]:
# dataframe with population sizes for each strata
strataN = pd.read_csv('definitions/strata-populations.tsv', sep='\t')
strataN['total'] = strataN[strataN.columns[1:]].sum(axis=1)
strataN.head()

,project_group,10-29,30-149,150-599,600-1199,1200+,total
0,arwiki,25,60,60,29,98,272
1,asia_wps,7,51,77,49,130,314
2,cee_wps,115,477,596,302,950,2440
3,commons,116,440,516,308,1489,2869
4,dewiki,80,505,703,335,851,2474


In [5]:
project_target_sizes = pd.DataFrame([strataN['project_group'],
                                     strataN['total']
                                     .apply(lambda N: nO/(1+(nO-1)/N))]).transpose()
project_target_sizes.head()

,project_group,total
0,arwiki,133.0116
1,asia_wps,142.282318
2,cee_wps,234.515943
3,commons,237.922113
4,dewiki,234.824922


### Step 3: Dividing the sample size across various edit bins

In [6]:
# dataframe with percentage distribution of 
bins_labels = strataN.columns[1:-1]
strataN_perc = strataN.copy()
strataN_perc[bins_labels] = (strataN_perc[bins_labels]
                             .div(strataN_perc[bins_labels].sum(axis=1), axis=0)
                             .multiply(100))
strataN_perc.drop('total', axis=1, inplace=True)
strataN_perc.head()

,project_group,10-29,30-149,150-599,600-1199,1200+
0,arwiki,9.191176,22.058824,22.058824,10.661765,36.029412
1,asia_wps,2.229299,16.242038,24.522293,15.605096,41.401274
2,cee_wps,4.713115,19.549180,24.426230,12.377049,38.934426
3,commons,4.043221,15.336354,17.985361,10.735448,51.899617
4,dewiki,3.233630,20.412288,28.415521,13.540825,34.397736


In [7]:
# sampling target sizes by each strata (projects and edit bins)
strata_target_sizes = strataN_perc.copy()
strata_target_sizes[bins_labels] = (strata_target_sizes[bins_labels].multiply(0.01)
                                    .mul(project_target_sizes.total.values, axis=0)
                                    .astype(float))
strata_target_sizes.head()

,project_group,10-29,30-149,150-599,600-1199,1200+
0,arwiki,12.225331,29.340794,29.340794,14.181384,47.923297
1,asia_wps,3.171899,23.109548,34.890887,22.203292,58.906692
2,cee_wps,11.053006,45.845945,57.283403,29.026154,91.307437
3,commons,9.619716,36.488578,42.791150,25.542004,123.480664
4,dewiki,7.593369,47.933139,66.726726,31.797231,80.774458


### Step 4: Adjust sampling target sizes to account for non-response

In [8]:
# response rates for each strata; provided GDI based on the previous cycle of the survey
strata_resp_rates = pd.read_csv('secrets/strata-response-rates.tsv', sep='\t').drop('total', axis=1)
strata_resp_rates.head()

,project_group,10-29,30-149,150-599,600-1199,1200+
0,arwiki,0.083333,0.038462,0.034483,0.102564,0.042373
1,asia_wps,0.000000,0.000000,0.080645,0.085714,0.094595
2,cee_wps,0.018519,0.026616,0.046745,0.047337,0.064732
3,commons,0.030488,0.071550,0.075812,0.097222,0.092792
4,dewiki,0.111111,0.097938,0.103841,0.095679,0.123750


In [9]:
strata_sample_pull = strata_resp_rates.copy()

for index in strata_sample_pull.index:
    for e_bin in bins_labels:
        
        # if the response rate is 0; then the population is multiplied by 0.057 to get the target
        if strata_resp_rates.loc[index, e_bin] == 0:
            strata_target = strataN.loc[index, 'total']*0.057
            
            # if a strata's target size is greater than the population size
            # then the target size is capped at population size
            if strata_target > strataN.loc[index, e_bin]:
                strata_sample_pull.loc[index, e_bin] = strataN.loc[index, e_bin]
            else:
                strata_sample_pull.loc[index, e_bin] = strata_target
        
        else:
            # adjusting the sampling target according the response rate
            strata_target = strata_target_sizes.loc[index, e_bin]/strata_resp_rates.loc[index, e_bin]
            
            # if a strata's target size is greater than the population size
            # then the target size is capped at population size            
            if  strata_target > strataN.loc[index, e_bin]:
                strata_sample_pull.loc[index, e_bin] = strataN.loc[index, e_bin]
            else:
                strata_sample_pull.loc[index, e_bin] = strata_target

strata_sample_pull[bins_labels] = strata_sample_pull[bins_labels].apply(np.ceil).astype(int)
strata_sample_pull

,project_group,10-29,30-149,150-599,600-1199,1200+
0,arwiki,25,60,60,29,98
1,asia_wps,7,18,77,49,130
2,cee_wps,115,477,596,302,950
3,commons,116,440,516,263,1331
4,dewiki,69,490,643,333,653
5,enwiki,536,1341,852,369,658
6,eswiki,80,312,369,151,361
7,frwiki,115,522,559,237,642
8,itwiki,39,227,255,103,326
9,jawiki,99,514,440,136,237


In [10]:
strata_sample_pull.to_csv('definitions/strata-sample-targets.tsv', sep='\t', index=False)